In [1]:
import yaml
import json
import sys
from pathlib import Path
from typing import Dict
import rdflib


In [2]:
def dict2yaml(path: str, data: Dict):
    with open(path, 'w') as yaml_f:
        yaml.safe_dump(data=data, stream=yaml_f)

In [ ]:
#example dict to use for copy and pasting
{'wikidata': {'uri': 'https://www.wikidata.org/wiki/Property:P582',
              'label': 'end_time'},
 'openresearch': {'uri': 'https://www.openresearch.org/wiki/Property:Wikidataid', 
                   'label': 'Wikidataid'},
 'gnd': {'uri': 'https://d-nb.info/standards/elementset/gnd#gndIdentifier', 
                   'label': 'GND-Identifier'}
}

With the below cell, we can create a valid JSON string by pasting the *uri* and *label* of the property we want to map to in a Python dict. It will then be made into the JSON string with the json.dumps() function and outputed in the output cell, from which it can be copied and pasted into Protégé.

In [85]:
# create mapping as python dict for each relevant property and print as JSON string to be pasted in Protégé as literal
maps_to = {'wikidata': {'uri': None,
                        'label': 'itemID'},
           'openresearch': {'uri': 'https://www.openresearch.org/wiki/Property:Wikidataid', 
                   'label': 'Wikidataid'}
          }
maps_to_annotation = json.dumps(maps_to)
maps_to_annotation

'{"wikidata": {"uri": null, "label": "itemID"}, "openresearch": {"uri": "https://www.openresearch.org/wiki/Property:Wikidataid", "label": "Wikidataid"}}'

In [82]:
# open a graph
graph = rdflib.Graph()
# load some data
graph.parse('C:/Users/StroemertP/AEON/aeon/aeon.ttl', format="ttl")  
# query for mapping dictionary in aeon:mapsTo
qres = graph.query(
    """
    PREFIX aeon: <https://github.com/tibonto/aeon#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    
    SELECT DISTINCT ?aeon_property ?maps_to ?aeon_property_domain ?rdfs_label
    
    WHERE {
            ?aeon_property aeon:AEON_0000026 ?maps_to.
            {?aeon_property rdf:type owl:ObjectProperty.} UNION             
            {?aeon_property rdf:type owl:DatatypeProperty.}
            OPTIONAL {?aeon_property rdfs:domain ?aeon_property_domain.}
            OPTIONAL {?aeon_property rdfs:label ?rdfs_label.}
           }

    """)

print("the graph has %s statements" % len(qres))


mapping_dict = {}
for printout in qres:
    printout_dict = printout.asdict()
    #print (printout_dict)
    
    # get the annotated aeon property
    aeon_property = str(printout_dict.get('aeon_property'))
    # cut off base URI
    aeon_property = aeon_property.replace("https://github.com/tibonto/aeon#","")
    
    # get the mappings from annotation property maps_to
    maps_to = json.loads(str(printout_dict.get('maps_to')))

    # get domain from annotation property or literal
    domain = str(printout_dict.get('aeon_property_domain'))

    mapping_dict[aeon_property] = {'maps_to': maps_to}
    mapping_dict[aeon_property]['domain'] = domain
   
    # code to parse all mapped plattforms here, this needs to be fleshed out!
    dict2yaml('confident_mapping.yml',mapping_dict)


the graph has 15 statements
